<a href="https://colab.research.google.com/github/chenboju/AI/blob/main/16_gensim_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2

--2024-05-20 05:54:39--  https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233419414 (223M) [application/octet-stream]
Saving to: ‘zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20240501-pag 100%[===================>] 222.61M  4.46MB/s    in 54s     

2024-05-20 05:55:33 (4.13 MB/s) - ‘zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2’ saved [233419414/233419414]



### opencc是繁簡轉換工具

In [2]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 6.5 MB/s eta 0:00:00


### gensim是訓練word2vec的函式庫

In [3]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [4]:
wiki_corpus

In [5]:
next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '而現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [6]:
text_num = 0  # 初始化文本計數器，用於跟踪處理的文本數量

# 打開名為 'wiki_text.txt' 的文本文件，使用 utf-8 編碼
# 並使用迴圈將每個文本寫入文件中
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        # 將每個文本的詞彙列表以空格分隔，並寫入文件中
        f.write(' '.join(text) + '\n')

        # 每處理完 10000 篇文章就打印出進度信息
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    # 最後再次打印處理的總文章數
    print('{} articles processed.'.format(text_num))


10000 articles processed.
20000 articles processed.
30000 articles processed.
32786 articles processed.


OpenCC: 用於將文本從簡體轉換為繁體，使用了 s2t 的模式。<br>
jieba: 用於對文本進行分詞。<br>
read(): 讀取文件中的文本。<br>
lcut(): 對文本進行分詞。<br>
過濾空字符: 通過列表推導式過濾掉分詞結果中的空字符。<br>
join(): 將分詞結果連接成一個字符串，並用空格分隔。<br>
write(): 將處理後的文本寫入到 seg.txt 文件中。<br>

In [7]:
import jieba  # 引入 jieba 库，用于中文分词
from opencc import OpenCC  # 引入 OpenCC 库，用于簡繁轉換

# 初始化 OpenCC，設置從簡體到繁體的轉換
cc = OpenCC('s2t')

# 讀取文件中的訓練數據，並將其進行簡體轉換
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)

# 使用 jieba 對文本進行分詞
train_data = jieba.lcut(train_data)

# 過濾空字符
train_data = [word for word in train_data if word != '']

# 將分詞結果連接成字符串，並用空格分隔
train_data = ' '.join(train_data)

# 將處理後的文本寫入到文件 'seg.txt' 中
open('seg.txt', 'w', encoding='utf-8').write(train_data)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


136623985

In [8]:
from gensim.models import word2vec

# 設置參數
seed = 666  # 隨機種子
sg = 0  # 使用 CBOW 模型
window_size = 10  # 上下文窗口大小
# vector_size = 100  # 詞向量的維度
min_count = 1  # 最小詞頻閾值，小於這個值的詞將會被忽略
workers = 8  # 訓練時使用的線程數
# epochs = 5  # 訓練的迭代次數
batch_words = 10000  # 每次訓練時的單詞數

# 從 'seg.txt' 文件中讀取訓練數據
train_data = word2vec.LineSentence('seg.txt')

# 使用 Word2Vec 模型進行訓練
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    # size=vector_size,
    workers=workers,
    # iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

# 將訓練好的模型保存到文件中
model.save('word2vec.model')


In [9]:
from gensim.models import word2vec

# 載入訓練好的詞向量模型
model = word2vec.Word2Vec.load('word2vec.model')

# 要查找相似詞彙的詞彙
string = '電腦'

# 輸出要查找相似詞彙的詞彙
print(string)

# 使用模型的 most_similar 方法查找與指定詞彙最相似的詞彙
for item in model.wv.most_similar(string):
    print(item)


電腦
('計算機', 0.7695195078849792)
('pda', 0.7647251486778259)
('軟體', 0.7627525329589844)
('ibm', 0.7318840026855469)
('程式', 0.7282395362854004)
('硬體', 0.7276607751846313)
('家用', 0.7192538976669312)
('遊戲機', 0.7177554965019226)
('模擬器', 0.7122527956962585)
('pc', 0.7100153565406799)
